# Modelling a gallium arsenide surface

This example shows how to use the
[atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html),
or ASE for short,
to set up and run a particular calculation of a gallium arsenide surface.
ASE is a Python package to simplify the process of setting up,
running and analysing results from atomistic simulations across different simulation codes.
By means of [ASEconvert](https://github.com/mfherbst/ASEconvert.jl) it is seamlessly
integrated with the AtomsBase ecosystem and thus available to DFTK via our own
AtomsBase integration.

In this example we will consider modelling the (1, 1, 0) GaAs surface separated by vacuum.

Parameters of the calculation. Since this surface is far from easy to converge,
we made the problem simpler by choosing a smaller `Ecut` and smaller values
for `n_GaAs` and `n_vacuum`.
More interesting settings are `Ecut = 15` and `n_GaAs = n_vacuum = 20`.

In [1]:
miller = (1, 1, 0)   # Surface Miller indices
n_GaAs = 2           # Number of GaAs layers
n_vacuum = 4         # Number of vacuum layers
Ecut = 5             # Hartree
kgrid = (4, 4, 1);   # Monkhorst-Pack mesh

Use ASE to build the structure:

In [2]:
using ASEconvert

a = 5.6537  # GaAs lattice parameter in Ångström (because ASE uses Å as length unit)
gaas = ase.build.bulk("GaAs", "zincblende"; a)
surface = ase.build.surface(gaas, miller, n_GaAs, 0, periodic=true);

Get the amount of vacuum in Ångström we need to add

In [3]:
d_vacuum = maximum(maximum, surface.cell) / n_GaAs * n_vacuum
surface = ase.build.surface(gaas, miller, n_GaAs, d_vacuum, periodic=true);

Write an image of the surface and embed it as a nice illustration:

In [4]:
ase.io.write("surface.png", surface * pytuple((3, 3, 1)), rotation="-90x, 30y, -75z")

Python: None

<img src="https://docs.dftk.org/stable/surface.png" width=500 height=500 />

Use the `pyconvert` function from `PythonCall` to convert to an AtomsBase-compatible system.
These two functions not only support importing ASE atoms into DFTK,
but a few more third-party data structures as well.
Typically the imported `atoms` use a bare Coulomb potential,
such that appropriate pseudopotentials need to be attached in a post-step:

In [5]:
using DFTK
system = attach_psp(pyconvert(AbstractSystem, surface);
                    Ga="hgh/pbe/ga-q3.hgh",
                    As="hgh/pbe/as-q5.hgh")

FlexibleSystem(As₂Ga₂, periodic = TTT):
    bounding_box      : [ 3.99777        0        0;
                                0  3.99777        0;
                                0        0  21.2014]u"Å"

    Atom(Ga, [       0,        0,  8.48055]u"Å")
    Atom(As, [ 3.99777,  1.99888,  9.89397]u"Å")
    Atom(Ga, [ 1.99888,  1.99888,  11.3074]u"Å")
    Atom(As, [ 1.99888, 6.96512e-16,  12.7208]u"Å")

      .---------.  
     /|         |  
    * |         |  
    | |         |  
    | |         |  
    | |         |  
    | |         |  
    | |         |  
    | |         |  
    | |         |  
    | |         |  
    | |         |  
    | |  As     |  
    | |   Ga    |  
    | |         |  
    | |        As  
    | |         |  
    | |         |  
    Ga|         |  
    | |         |  
    | |         |  
    | |         |  
    | |         |  
    | |         |  
    | |         |  
    | |         |  
    | .---------.  
    |/         /   
    *---------*    


We model this surface with (quite large a) temperature of 0.01 Hartree
to ease convergence. Try lowering the SCF convergence tolerance (`tol`)
or the `temperature` or try `mixing=KerkerMixing()`
to see the full challenge of this system.

In [6]:
model = model_DFT(system, [:gga_x_pbe, :gga_c_pbe],
                  temperature=0.001, smearing=DFTK.Smearing.Gaussian())
basis = PlaneWaveBasis(model; Ecut, kgrid)

scfres = self_consistent_field(basis, tol=1e-4, mixing=LdosMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.58703617479                   -0.58    5.3         
  2   -16.72517035967       -0.86       -1.01    1.0    277ms
  3   -16.73060128664       -2.27       -1.57    2.1    300ms
  4   -16.73127525906       -3.17       -2.16    2.0    296ms
  5   -16.73133263838       -4.24       -2.61    2.2    376ms
  6   -16.73133482018       -5.66       -2.95    1.6    253ms
  7   -16.73097346007   +   -3.44       -2.53    2.3    346ms
  8   -16.73133570125       -3.44       -3.26    2.4    309ms
  9   -16.73133715896       -5.84       -3.47    2.2    331ms
 10   -16.73133757162       -6.38       -3.58    2.4    319ms
 11   -16.73133876857       -5.92       -3.70    2.3    327ms
 12   -16.73134016803       -5.85       -4.51    2.3    314ms


In [7]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             5.8593798 
    AtomicLocal         -105.6098987
    AtomicNonlocal      2.3494717 
    Ewald               35.5044300
    PspCorrection       0.2016043 
    Hartree             49.5613300
    Xc                  -4.5976537
    Entropy             -0.0000035

    total               -16.731340168034